In [1]:
# import datetime
# import cv2
# import os
# import matplotlib.pyplot as plt
# from collections import defaultdict
# from ultralytics import YOLO
# from deep_sort_realtime.deepsort_tracker import DeepSort

# CONFIDENCE_THRESHOLD = 0.5
# GREEN = (0, 255, 0)
# WHITE = (255, 255, 255)
# RED = (0, 0, 255)

# # 클래스 정보 입력
# with open('/Users/kimhyeonjeong/Documents/2024-2/OD/code/yolo_class.txt', 'r') as coco128:
#     class_list = coco128.read().split('\n')

# # YOLO 모델과 DeepSort 초기화
# model = YOLO('/Users/kimhyeonjeong/Documents/2024-2/OD/code/best.pt')
# tracker = DeepSort(max_age=50)

# # 이미지 폴더 설정
# image_folder_path = '/Users/kimhyeonjeong/Documents/2024-2/OD/code/bel12'  # 이미지 폴더 경로
# file_names = os.listdir(image_folder_path)
# image_files = [f for f in file_names if f.endswith(('.jpg', '.png'))]
# image_files.sort()

# # 트래킹 ID 및 라이트 RGB 기록을 저장할 딕셔너리 초기화
# class_track_ids = {}
# light_rgb_history = defaultdict(list)

# # 비디오 저장 설정
# output_video_path = '/Users/kimhyeonjeong/Documents/2024-2/OD/code/output/bel12_output.mp4'
# frame_width, frame_height = cv2.imread(os.path.join(image_folder_path, image_files[0])).shape[1], cv2.imread(os.path.join(image_folder_path, image_files[0])).shape[0]
# out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), 30.0, (frame_width, frame_height))

# # 이미지 순차 처리
# frame_count = 0
# for image_file in image_files:
#     image_path = os.path.join(image_folder_path, image_file)
#     frame = cv2.imread(image_path)
#     if frame is None:
#         print(f"Warning: Could not load image {image_file}")
#         continue

#     # 프레임 처리 시작 시간
#     start = datetime.datetime.now()

#     # YOLO 모델을 사용하여 객체 감지 수행
#     detection = model.predict(source=[frame], save=False)[0]
#     results = []

#     for data in detection.boxes.data.tolist():  # data : [xmin, ymin, xmax, ymax, confidence_score, class_id]
#         confidence = float(data[4])
#         if confidence < CONFIDENCE_THRESHOLD:
#             continue

#         xmin, ymin, xmax, ymax = int(data[0]), int(data[1]), int(data[2]), int(data[3])
#         width = xmax - xmin
#         height = ymax - ymin
#         label = int(data[5])
#         class_name = class_list[label]

#         # 클래스별 트래킹 ID 관리
#         if class_name not in class_track_ids:
#             class_track_ids[class_name] = {}

#         bbox = (xmin, ymin, width, height)
#         results.append([bbox, confidence, label])

#     # DeepSort를 사용하여 트래킹 업데이트
#     tracks = tracker.update_tracks(results, frame=frame)

#     # 자동차 및 라이트 바운딩 박스 추출
#     car_boxes = []
#     light_boxes = []
#     light_ids_rgb = []
#     for track in tracks:
#         if not track.is_confirmed():
#             continue

#         track_id = track.track_id
#         ltrb = track.to_ltrb()
#         label = track.det_class
#         class_name = class_list[label]

#         # 트래킹 ID 할당
#         if track_id not in class_track_ids[class_name]:
#             class_track_ids[class_name][track_id] = f"{class_name}-{len(class_track_ids[class_name]) + 1}"

#         assigned_id = class_track_ids[class_name][track_id]

#         xmin, ymin, xmax, ymax = int(ltrb[0]), int(ltrb[1]), int(ltrb[2]), int(ltrb[3])
#         label_text = f"{assigned_id}"

#         if class_name == "car":
#             car_boxes.append((xmin, ymin, xmax, ymax))
#         elif class_name == "light":
#             light_boxes.append((xmin, ymin, xmax, ymax))
#             # 라이트 바운딩 박스에서 RGB 값 추출 및 트래킹이 시작된 경우만 기록
#             if track.is_confirmed() and track.time_since_update == 0:
#                 light_region = frame[ymin:ymax, xmin:xmax]
#                 avg_color = cv2.mean(light_region)[:3]  # BGR을 RGB로 변환
#                 light_rgb = (int(avg_color[2]), int(avg_color[1]), int(avg_color[0]))
#                 light_ids_rgb.append((assigned_id, light_rgb))

#             # RGB 값 기록
#             if not light_rgb_history[assigned_id]:
#                 light_rgb_history[assigned_id] = [(None, None, None)] * frame_count
#             light_rgb_history[assigned_id].append(light_rgb)
            
#         # 바운딩 박스 및 라벨 그리기
#         cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), box_color, 2)
#         cv2.putText(frame, label_text, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, WHITE, 2)

#     # FPS 계산 및 표시
#     end = datetime.datetime.now()
#     total = (end - start).total_seconds()
#     fps = f'FPS: {1 / total:.2f}'
#     cv2.putText(frame, f'FPS: {1 / total:.2f} (Time: {frame_count / 30:.2f}s)', (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

#     # 프레임 수 표시
#     cv2.putText(frame, f'Frame: {frame_count} (Time: {frame_count / 30:.2f}s)', (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)

#     # 결과 프레임 저장
#     out.write(frame)

#     # 결과 프레임 표시
#     cv2.imshow('frame', frame)

#     if cv2.waitKey(1) == ord('q'):
#         break

#     frame_count += 1

# # 비디오 저장 종료
# out.release()
# cv2.destroyAllWindows()

In [2]:
# # Plot average RGB values and variance of average RGB values for each light
# import numpy as np
# import matplotlib.pyplot as plt

# light_rgb_history = {k: v for k, v in light_rgb_history.items() if v}  # Keep only lights that have tracking history
# num_lights = len(light_rgb_history)
# fig, axes = plt.subplots(num_lights, 4, figsize=(24, 6 * num_lights), sharex=True)
# if num_lights == 1:
#     axes = [axes]

# # Calculate the global maximum variance to set y-axis limits consistently across all variance plots
# global_max_variance = max(
#     [
#         np.var([val for val in [(r, g, b) if (r, g, b) != (None, None, None) else None for r, g, b in rgb_values][:i] if val is not None])
#         for rgb_values in light_rgb_history.values()
#         for i in range(1, len(rgb_values) + 1)
#         if len([val for val in [(r, g, b) if (r, g, b) != (None, None, None) else None for r, g, b in rgb_values][:i] if val is not None]) > 1
#     ],
#     default=1
# )

# for (light_id, rgb_values), ax_pair in zip(light_rgb_history.items(), axes):
#     ax_rgb, ax_diff_rg_rb, ax_avg, ax_avg_var = ax_pair

#     # Plot RGB values
#     r_values = [r if (r, g, b) != (None, None, None) else None for r, g, b in rgb_values]
#     g_values = [g if (r, g, b) != (None, None, None) else None for r, g, b in rgb_values]
#     b_values = [b if (r, g, b) != (None, None, None) else None for r, g, b in rgb_values]
#     ax_rgb.plot(range(len(r_values)), r_values, label=f'{light_id} - R', color='r')
#     ax_rgb.plot(range(len(g_values)), g_values, label=f'{light_id} - G', color='g')
#     ax_rgb.plot(range(len(b_values)), b_values, label=f'{light_id} - B', color='b')
#     ax_rgb.set_xlabel('Frame')
#     ax_rgb.set_ylabel('RGB Value')
#     ax_rgb.set_ylim(0, 255)  # Set y-axis limit to ensure consistent scaling for RGB values
#     ax_rgb.set_title(f'RGB Values Over Frames for {light_id}')
#     ax_rgb.legend()

#     # Calculate R-G and R-B differences
#     rg_diff = [r - g if (r is not None and g is not None) else None for r, g in zip(r_values, g_values)]
#     rb_diff = [r - b if (r is not None and b is not None) else None for r, b in zip(r_values, b_values)]

#     # Plot R-G and R-B differences
#     ax_diff_rg_rb.plot(range(len(rg_diff)), rg_diff, label=f'{light_id} - R-G', color='purple')
#     ax_diff_rg_rb.plot(range(len(rb_diff)), rb_diff, label=f'{light_id} - R-B', color='orange')
#     ax_diff_rg_rb.set_xlabel('Frame')
#     ax_diff_rg_rb.set_ylabel('Difference Value')
#     ax_diff_rg_rb.set_ylim(min([val for val in (rg_diff + rb_diff) if val is not None]) - 10, max([val for val in (rg_diff + rb_diff) if val is not None]) + 10)  # Set y-axis limit to ensure consistent scaling for difference values
#     ax_diff_rg_rb.set_title(f'R-G and R-B Differences Over Frames for {light_id}')
#     ax_diff_rg_rb.legend()

#     # Calculate average RGB values per frame
#     avg_rgb_values = [(r + g + b) / 3 if (r, g, b) != (None, None, None) else None for r, g, b in rgb_values]
    
#     # Plot average RGB values
#     ax_avg.plot(range(len(avg_rgb_values)), avg_rgb_values, label=f'{light_id} - Avg RGB', color='b')
#     ax_avg.set_xlabel('Frame')
#     ax_avg.set_ylabel('Average RGB Value')
#     ax_avg.set_ylim(0, 255)  # Set y-axis limit to ensure consistent scaling for average RGB values
#     ax_avg.set_title(f'Average RGB Values Over Frames for {light_id}')
#     ax_avg.legend()

#     # Calculate variance of average RGB values over the frames
#     avg_variances = []
#     for i in range(1, len(avg_rgb_values) + 1):
#         valid_values = [val for val in avg_rgb_values[i-10:i+11] if val is not None]
#         if len(valid_values) > 1:
#             avg_variances.append(np.var(valid_values))
#         else:
#             avg_variances.append(None)
    
#     # Plot variance of average RGB values with y-axis scaled to the global maximum variance
#     ax_avg_var.plot(range(len(avg_variances)), avg_variances, label=f'{light_id} - Variance of Avg RGB', color='c')
#     ax_avg_var.set_ylim(0, global_max_variance+20)  # Set y-axis limit to the global maximum variance value
#     ax_avg_var.set_xlabel('Frame')
#     ax_avg_var.set_ylabel('Variance')
#     ax_avg_var.set_title(f'Variance of Average RGB Values Over Frames for {light_id}')
#     ax_avg_var.legend()

# plt.tight_layout()
# plt.show()

##  on-off 알고리즘 적용

In [8]:
import datetime
import cv2
import os
import matplotlib.pyplot as plt
from collections import defaultdict
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

CONFIDENCE_THRESHOLD = 0.5
GREEN = (0, 255, 0)
WHITE = (255, 255, 255)
RED = (0, 0, 255)

# 클래스 정보 입력
with open('/Users/kimhyeonjeong/Documents/2024-2/OD/code/yolo_class.txt', 'r') as coco128:
    class_list = coco128.read().split('\n')

# YOLO 모델과 DeepSort 초기화
model = YOLO('/Users/kimhyeonjeong/Documents/2024-2/OD/code/best.pt')
tracker = DeepSort(max_age=50)

# 이미지 폴더 설정
image_folder_path = '/Users/kimhyeonjeong/Documents/2024-2/OD/code/bel12'  # 이미지 폴더 경로
file_names = os.listdir(image_folder_path)
image_files = [f for f in file_names if f.endswith(('.jpg', '.png'))]
image_files.sort()

# 트래킹 ID 및 라이트 RGB 기록을 저장할 딕셔너리 초기화
class_track_ids = {}
light_rgb_history = defaultdict(list)

# 비디오 저장 설정
output_video_path = '/Users/kimhyeonjeong/Documents/2024-2/OD/code/output/bel12_output.mp4'
frame_width, frame_height = cv2.imread(os.path.join(image_folder_path, image_files[0])).shape[1], cv2.imread(os.path.join(image_folder_path, image_files[0])).shape[0]
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), 30.0, (frame_width, frame_height))




In [9]:
import numpy as np

def update_light_box_color(light_rgb_history, frame_count, assigned_id):
    # 10프레임 단위로 R-G, R-B 계산 및 분산/shift mean 계산
    if frame_count % 10 == 0 and len(light_rgb_history[assigned_id]) >= 10:
        last_10_frames = light_rgb_history[assigned_id][-10:]
        r_values = [rgb[0] for rgb in last_10_frames if rgb != (None, None, None)]
        g_values = [rgb[1] for rgb in last_10_frames if rgb != (None, None, None)]
        b_values = [rgb[2] for rgb in last_10_frames if rgb != (None, None, None)]

        if len(r_values) == 10:
            r_g_diff = [r - g for r, g in zip(r_values, g_values)]
            r_b_diff = [r - b for r, b in zip(r_values, b_values)]

            variance_r_g = np.var(r_g_diff)
            variance_r_b = np.var(r_b_diff)

            if variance_r_g > VARIANCE_THRESHOLD or variance_r_b > VARIANCE_THRESHOLD:
                return RED
            else:
                shift_mean_r = np.mean(np.diff(r_values))
                if shift_mean_r > SHIFT_MEAN_THRESHOLD:
                    return RED
                else:
                    return GREEN
    return GREEN

# Additional variables
THRESHOLD_DIFF = 50
VARIANCE_THRESHOLD = 100
SHIFT_MEAN_THRESHOLD = 30

# Initialize light_ids_rgb
light_ids_rgb = []

# Update color for bounding boxes based on R-G, R-B difference and variance
frame_count = 0
for image_file in image_files:
    image_path = os.path.join(image_folder_path, image_file)
    frame = cv2.imread(image_path)
    if frame is None:
        print(f"Warning: Could not load image {image_file}")
        continue

    # YOLO 모델을 사용하여 객체 감지 수행
    detection = model.predict(source=[frame], save=False)[0]
    results = []

    for data in detection.boxes.data.tolist():  # data : [xmin, ymin, xmax, ymax, confidence_score, class_id]
        confidence = float(data[4])
        if confidence < CONFIDENCE_THRESHOLD:
            continue

        xmin, ymin, xmax, ymax = int(data[0]), int(data[1]), int(data[2]), int(data[3])
        width = xmax - xmin
        height = ymax - ymin
        label = int(data[5])
        class_name = class_list[label]

        # 클래스별 트래킹 ID 관리
        if class_name not in class_track_ids:
            class_track_ids[class_name] = {}

        bbox = (xmin, ymin, width, height)
        results.append([bbox, confidence, label])

    # DeepSort를 사용하여 트래킹 업데이트
    tracks = tracker.update_tracks(results, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue

        track_id = track.track_id
        ltrb = track.to_ltrb()
        label = track.det_class
        class_name = class_list[label]

        if track_id not in class_track_ids[class_name]:
            class_track_ids[class_name][track_id] = f"{class_name}-{len(class_track_ids[class_name]) + 1}"

        assigned_id = class_track_ids[class_name][track_id]
        xmin, ymin, xmax, ymax = int(ltrb[0]), int(ltrb[1]), int(ltrb[2]), int(ltrb[3])
        label_text = f"{assigned_id}"

        box_color = GREEN
        if class_name == "light":
            light_region = frame[ymin:ymax, xmin:xmax]
            avg_color = cv2.mean(light_region)[:3]  # BGR to RGB
            light_rgb = (int(avg_color[2]), int(avg_color[1]), int(avg_color[0]))
            light_ids_rgb.append((assigned_id, light_rgb))

            # RGB 값 기록
            if not light_rgb_history[assigned_id]:
                light_rgb_history[assigned_id] = [(None, None, None)] * frame_count
            light_rgb_history[assigned_id].append(light_rgb)

            # Update box color using the function
            box_color = update_light_box_color(light_rgb_history, frame_count, assigned_id)

        # Draw bounding box and label
        cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), box_color, 2)
        cv2.putText(frame, label_text, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, WHITE, 2)

    # 결과 프레임 저장 및 표시
    out.write(frame)
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) == ord('q'):
        break

    frame_count += 1

# 비디오 저장 종료
out.release()
cv2.destroyAllWindows()



0: 384x640 2 Cars, 4 Lights, 176.0ms
Speed: 1.6ms preprocess, 176.0ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Cars, 4 Lights, 165.5ms
Speed: 1.5ms preprocess, 165.5ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Cars, 4 Lights, 202.5ms
Speed: 1.3ms preprocess, 202.5ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Cars, 4 Lights, 170.2ms
Speed: 1.4ms preprocess, 170.2ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Cars, 4 Lights, 204.1ms
Speed: 1.6ms preprocess, 204.1ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Cars, 4 Lights, 168.8ms
Speed: 1.3ms preprocess, 168.8ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Cars, 4 Lights, 177.4ms
Speed: 2.0ms preprocess, 177.4ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Cars, 4 Lights, 182.2ms
Speed: 1.6